Kaggle 폴더에다가 kaggle.json 토큰 갖다 넣고 시작해라~~~~~~~~~

In [ ]:
import os
!mkdir Kaggle
os.environ['KAGGLE_CONFIG_DIR'] = "/content/Kaggle"

In [ ]:
#changing the working directory
%cd /content/Kaggle
#Check the present working directory using pwd command

/content/Kaggle


In [ ]:
!kaggle datasets download -d chrisfilo/urbansound8k

100% 5.60G/5.61G [01:57<00:00, 76.2MB/s]
100% 5.61G/5.61G [01:57<00:00, 51.5MB/s]


In [ ]:
!ls

kaggle.json  urbansound8k.zip


In [ ]:
!mkdir folds
!unzip \*.zip && rm *.zip
!rm -rf fold2/
!rm -rf fold4/
!rm -rf fold5/
!rm -rf fold6/
!rm -rf fold8/
!rm -rf fold10/

!mv fold1 /content/Kaggle/folds
!mv fold3 /content/Kaggle/folds
!mv fold7 /content/Kaggle/folds
!mv fold9 /content/Kaggle/folds

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: fold4/151877-5-1-0.wav  
  inflating: fold4/154758-5-0-0.wav  
  inflating: fold4/154758-5-0-1.wav  
  inflating: fold4/154758-5-0-10.wav  
  inflating: fold4/154758-5-0-11.wav  
  inflating: fold4/154758-5-0-12.wav  
  inflating: fold4/154758-5-0-13.wav  
  inflating: fold4/154758-5-0-14.wav  
  inflating: fold4/154758-5-0-15.wav  
  inflating: fold4/154758-5-0-16.wav  
  inflating: fold4/154758-5-0-17.wav  
  inflating: fold4/154758-5-0-18.wav  
  inflating: fold4/154758-5-0-19.wav  
  inflating: fold4/154758-5-0-2.wav  
  inflating: fold4/154758-5-0-20.wav  
  inflating: fold4/154758-5-0-21.wav  
  inflating: fold4/154758-5-0-3.wav  
  inflating: fold4/154758-5-0-4.wav  
  inflating: fold4/154758-5-0-5.wav  
  inflating: fold4/154758-5-0-6.wav  
  inflating: fold4/154758-5-0-7.wav  
  inflating: fold4/154758-5-0-8.wav  
  inflating: fold4/154758-5-0-9.wav  
  inflating: fold4/155320-3-0-0.wav  
  inflating: fold4/156362-4-0-0.wav  
  

여기까지 데이터 다운 및 정리

In [ ]:
import pandas as pd
import numpy as np

metadata = pd.read_csv('/content/Kaggle/UrbanSound8K.csv')

In [ ]:
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


업로드 한 후, 교수님이 말한 대로 사이즈 좀 줄이자

줄이긴 줄이되, 어떻게 줄인건지 => 비슷한 소리인 것들끼리 모아서 그걸 구별하게 하고,

데이터도 폴드 하나나 두개 정도만 쓰도록 하자
그것만 해도 많을 것 같긴 한데 지켜보되,

일단 4, 5, 7 클래스 = drilling, enginge_idling, jackhammer
얘네들이 서로 비슷한 소리.

그러면 4, 5, 7클래스만 뽑아서 보자고 각 폴드마다 숫자 비슷하게 있는 걸 골라보자
classID랑 fold는 정수값

fold 1,3,7,9의 4,5,7클래스만 뽑자! 400, 398, 398

In [ ]:
sliced = metadata[["slice_file_name","fold","classID"]]
drill = sliced[sliced["classID"]==4]
egn_idling = sliced[sliced["classID"]==5]
jkhmm = sliced[sliced["classID"]==7]
drl = pd.DataFrame()
egn_idl = pd.DataFrame()
jkhm = pd.DataFrame()
for i in [1,3,7,9]:
  drl = drl.append(drill[drill["fold"]==i])
for i in [1,3,7,9]:
  egn_idl = egn_idl.append(egn_idling[egn_idling["fold"]==i])
for i in [1,3,7,9]:
  jkhm = jkhm.append(jkhmm[jkhmm["fold"]==i])

#400, 398, 398 for each df

10-cross validation은 이제 할 필요 없고,
sklearn.model_selection.train_test_split()을 써서 나누자

시험해 볼 모델들 리스트 후보군 :

SVC

K-NN

Random Forest

Naive-Bayes

이정도가 있고 모델은,

X는 오디오 파일이고, y는 classID값이다




In [ ]:
# drl, egn_idl, jkhm 합치기
dataset = pd.DataFrame()
dataset = dataset.append(drl)
dataset = dataset.append(egn_idl)
dataset = dataset.append(jkhm)

In [ ]:
import librosa, librosa.display
# fold는 1,3,7,9
Z = []
DATADIR = '/content/Kaggle/folds'
def create_Z_list(dataset):
  folddict = {1:'fold1',3:'fold3',7:'fold7',9:'fold9'}
  for fold in [1,3,7,9]:
    path=os.path.join(DATADIR,folddict[fold]) #/dataset/fold1
    for audio in os.listdir(path):
      if audio in dataset.loc[dataset['classID'] == 4, ['slice_file_name']].to_numpy(): #if string in pandas.DataFrame
        audio_array,sr_array=librosa.load(os.path.join(path,audio))
        audio_array = audio_array[0:12789]
        class_num = dataset.loc[dataset['slice_file_name'] == audio, 'classID'].iloc[0]
        Z.append([audio_array,class_num])
      elif audio in dataset.loc[dataset['classID'] == 5, ['slice_file_name']].to_numpy(): #if string in pandas.DataFrame
        audio_array,sr_array=librosa.load(os.path.join(path,audio))
        audio_array = audio_array[0:12789]
        class_num = dataset.loc[dataset['slice_file_name'] == audio, 'classID'].iloc[0]
        Z.append([audio_array,class_num])
      elif audio in dataset.loc[dataset['classID'] == 7, ['slice_file_name']].to_numpy(): #if string in pandas.DataFrame
        audio_array,sr_array=librosa.load(os.path.join(path,audio))
        audio_array = audio_array[0:12789]
        class_num = dataset.loc[dataset['slice_file_name'] == audio, 'classID'].iloc[0]
        Z.append([audio_array,class_num])
create_Z_list(dataset) # 4분 걸림

In [ ]:
print(len(Z))
lenofaudio = len(Z)

1196


In [ ]:
X=[]
y=[]
for categories, label in Z:
  X.append(categories)
  y.append(label)

In [ ]:
type(X)#[array,array]
for i in range(lenofaudio):
  X[i] = X[i].tolist()

In [ ]:
X = np.vstack(X)
X = np.array(X).reshape(lenofaudio,-1)
X.shape

(1196, 12789)

In [ ]:
y=np.array(y)
y.shape

(1196,)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

SVC -> 디폴트가 예측값이 그나마 높게 나옴

In [ ]:
from sklearn.svm import SVC

svc = SVC()#gamma 는 디폴트 값 #kernel도 디폴트 값 디폴트 값들이 그나마 높게 나오네
svc.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [ ]:
y2 = svc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy score is", accuracy_score(y_test,y2))

Accuracy score is 0.26755852842809363


K-NN
->일반 < 5 cross validation < GridSearch 5 cross validation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [ ]:
knn.score(X_test, y_test)

0.3979933110367893

In [ ]:
from sklearn.model_selection import cross_val_score
knn_cv = KNeighborsClassifier()
cv_scores = cross_val_score (knn_cv, X_train, y_train, cv = 3) #3이 제일 좋은 퍼포먼스

In [ ]:
print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

[0.35785953 0.42140468 0.40802676]
cv_scores mean:0.3957636566332218


In [ ]:
from sklearn.model_selection import GridSearchCV
knn2 = KNeighborsClassifier()
param_grid = {'n_neighbors': np.arange(1, 5)}
knn_gscv = GridSearchCV(knn2, param_grid, cv=5)
knn_gscv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': array([1, 2, 3, 4])},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [ ]:
y3 = knn_gscv.predict(X_test)
print(knn_gscv.best_params_)

print(knn_gscv.best_score_)

print(knn_gscv.score(X_test, y_test))

{'n_neighbors': 2}
0.43144630664183736
0.46488294314381273


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF = RandomForestClassifier(criterion = 'entropy') #criterion = 'entropy'
RF.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [ ]:
y4 = RF.predict(X_test)
RF.score(X_test,y_test)

0.41471571906354515

Naive-Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB()
GNB.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [ ]:
y5 = GNB.predict(X_test)
GNB.score(X_test,y_test)

0.3745819397993311

Performance metrics

In [ ]:
from sklearn.metrics import classification_report
print("Classification report for SVC is")
print(classification_report(y_test,y2))
print("Classification report for k-NN GSCV is")
print(classification_report(y_test,y3))
print("Classification report for Random Forest is")
print(classification_report(y_test,y4))
print("Classification report for Gaussian Naive-Bayes is")
print(classification_report(y_test,y5))

Classification report for SVC is
              precision    recall  f1-score   support

           4       0.29      0.02      0.03       114
           5       0.26      0.63      0.37        90
           7       0.28      0.22      0.25        95

    accuracy                           0.27       299
   macro avg       0.28      0.29      0.22       299
weighted avg       0.28      0.27      0.20       299

Classification report for k-NN GSCV is
              precision    recall  f1-score   support

           4       0.49      0.61      0.54       114
           5       0.47      0.46      0.46        90
           7       0.41      0.29      0.34        95

    accuracy                           0.46       299
   macro avg       0.46      0.45      0.45       299
weighted avg       0.46      0.46      0.46       299

Classification report for Random Forest is
              precision    recall  f1-score   support

           4       0.41      0.25      0.31       114
           5  

precision: "긍정적인 결과를 예측할 때, 얼마나 자주 옳은가?"

recall: "실제로 긍정적인 결과일 때, 얼마나 자주 정확하게 예측할 수 있을까요?"

f1-score: 위 둘의 조화평균